In [1]:
import os
import sys
from typing import Optional, cast

import src.evals.data as data_module
import src.hf_bert as hf_bert_module
import src.mosaic_bert as mosaic_bert_module
import src.flex_bert as flex_bert_module
import transformers
from composer import Trainer, algorithms, Evaluator
from composer.callbacks import LRMonitor, MemoryMonitor, OptimizerMonitor, RuntimeEstimator, SpeedMonitor
from composer.core.types import Dataset
from composer.loggers import WandBLogger
from composer.optim import DecoupledAdamW
from composer.optim.scheduler import (
    ConstantWithWarmupScheduler,
    CosineAnnealingWithWarmupScheduler,
    LinearWithWarmupScheduler,
)
from src.scheduler import WarmupStableDecayScheduler
from composer.utils import dist, reproducibility
from omegaconf import DictConfig
from omegaconf import OmegaConf as om
from torch.utils.data import DataLoader

/home/goreski/miniconda/envs/bert24/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/goreski/miniconda/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:958: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/goreski/miniconda/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:1017: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [2]:
def build_my_dataloader(cfg: DictConfig, device_batch_size: int):
    """Create a dataloader for classification.

    **Modify this function to train on your own dataset!**

    This function is provided as a starter code to simplify fine-tuning a BERT
    classifier on your dataset. We'll use the dataset for QNLI (one of the
    GLUE tasks) as a demonstration.

    Args:
        cfg (DictConfig): An omegaconf config that houses all the configuration
            variables needed to instruct dataset/dataloader creation.
        device_batch_size (int): The size of the batches that the dataloader
            should produce.

    Returns:
        dataloader: A dataloader set up for use of the Composer Trainer.
    """
    # As a demonstration, we're using the QNLI dataset from the GLUE suite
    # of tasks.
    #
    # Note: We create our dataset using the `data_module.create_glue_dataset` utility
    #   defined in `./src/glue/data.py`. If you inspect that code, you'll see
    #   that we're taking some extra steps so that our dataset yields examples
    #   that follow a particular format. In particular, the raw text is
    #   tokenized and some of the data columns are removed. The result is that
    #   each example is a dictionary with the following:
    #
    #     - 'input_ids': the tokenized raw text
    #     - 'label': the target class that the text belongs to
    #     - 'attention_mask': a list of 1s and 0s to indicate padding
    #
    # When you set up your own dataset, it should handle tokenization to yield
    # examples with a similar structure!
    #
    # REPLACE THIS WITH YOUR OWN DATASET:
    dataset = data_module.create_glue_dataset(
        task="qnli",
        split=cfg.split,
        tokenizer_name=cfg.tokenizer_name,
        max_seq_length=cfg.max_seq_len,
    )

    dataset = cast(Dataset, dataset)
    dataloader = DataLoader(
        dataset,
        # As an alternative to formatting the examples inside the dataloader,
        # you can write a custom data collator to do that instead.
        collate_fn=transformers.default_data_collator,
        batch_size=device_batch_size,
        sampler=dist.get_sampler(dataset, drop_last=cfg.drop_last, shuffle=cfg.shuffle),
        num_workers=cfg.num_workers,
        pin_memory=cfg.get("pin_memory", True),
        prefetch_factor=cfg.get("prefetch_factor", 2),
        persistent_workers=cfg.get("persistent_workers", True),
        timeout=cfg.get("timeout", 0),
    )

    return dataloader

yaml_path, args_list = "yamls/test/sequence_classification.yaml", sys.argv[2:]
with open("yamls/defaults.yaml") as f:
    default_cfg = om.load(f)
with open(yaml_path) as f:
    yaml_cfg = om.load(f)
cli_cfg = om.from_cli(args_list)
cfg = om.merge(default_cfg, yaml_cfg, cli_cfg)
cfg = cast(DictConfig, cfg)  # for type checking

train_loader = build_my_dataloader(
    cfg.train_loader,
    cfg.global_train_batch_size // dist.get_world_size(),
)

#get one data sample from the train_loader
data = next(iter(train_loader))
# Print data keys only
print(data.keys())
print(data)

/home/goreski/miniconda/envs/bert24/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Example from glue dataset:
{'question': 'When did the third Digimon series begin?', 'sentence': 'Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese.', 'label': 1, 'idx': 0}
{'question': 'Which missile batteries often have individual launchers several kilometres from one another?', 'sentence': 'When MANPADS is operated by specialists, batteries may have several dozen teams deploying separately in small sections; self-propelled air defence guns may deploy in pairs.', 'label': 1, 'idx': 1}
{'question': "What two things does Popper argue Tarski's theory involves in an evaluation of truth?", 'sentence': 'He bases this interpretation on the fact that examples such as the one described above refer to two things: assertions and the facts to which they refer.', 'label': 0, 'idx': 2}

In [7]:
import transformers
from generate_dataset import generate_synthetic_dataset

# Configuration parameters
decimal_token = "<DEC>"

cfg = {}

# Generate the synthetic dataset
df = generate_synthetic_dataset(
    n_samples=cfg.get("n_samples", 100),
    n_continuous_features=cfg.get("n_continuous_features", 15),
    n_discrete_features=cfg.get("n_discrete_features", 15),
    n_classes=cfg.get("n_classes", 2),
    class_distribution=cfg.get("class_distribution", [0.8, 0.2]),
    n_bins=cfg.get("n_bins", 10),
    n_redundant=cfg.get("n_redundant", 5),
    n_noisy=cfg.get("n_noisy", 20),
    class_sep=cfg.get("class_sep", 0.1),
)

# Change structure to "sentence", "label" and "idx"
# All columns except the last one are features and they are concatenated to form a sentence
# The last column is the label
df['sentence'] = df.drop(columns=['label']).apply(lambda x: ' '.join([f"{val}".replace('.', decimal_token) for val in x]), axis=1)

# Create dummy sentence based on label: if 1 then "4.23245", if 0 then "5.7655"
df['sentence'] = df['label'].apply(lambda x: f"4{decimal_token}23245" if x == 1 else f"5{decimal_token}7655")

# Reorder columns and add index
df = df[['sentence', 'label']]
df['idx'] = df.index

# Tokenize the dataset
tokenizer = transformers.AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

# Add special tokens
tokenizer.add_tokens([decimal_token])

# Add a padding token if it doesn't already exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token_id = tokenizer.eos_token_id  # Use EOS token as padding token for GPT-2

# Tokenize sentences
tokenized_dataset = tokenizer(
    df['sentence'].tolist(),  # Ensure this is a list of strings
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

# Print input sentence and tokenization results
print("\nTokenizer Debug Info:")
print("-" * 50)
# Print first 3 examples
for i in range(min(3, len(df))):
    print(f"\nExample {i+1}:")
    print(f"Input sentence: {df['sentence'].iloc[i]}")
    print(f"Label: {df['label'].iloc[i]}")
    
    # Get tokenized ids for this example
    tokens = tokenizer.encode(df['sentence'].iloc[i])
    print(f"Token IDs: {tokens}")
    
    # Decode back to string to verify tokenization
    decoded = tokenizer.decode(tokens)
    print(f"Decoded text: {decoded}")
    
    # Print individual tokens
    tokens_list = tokenizer.convert_ids_to_tokens(tokens)
    print(f"Individual tokens: {tokens_list}")
print("-" * 50)

# Create a PyTorch dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

labels = df['label'].tolist()
custom_dataset = CustomDataset(tokenized_dataset, labels)


Tokenizer Debug Info:
--------------------------------------------------

Example 1:
Input sentence: 5<DEC>7655
Label: 0
Token IDs: [101, 1019, 30522, 6146, 24087, 102]
Decoded text: [CLS] 5 <DEC> 7655 [SEP]
Individual tokens: ['[CLS]', '5', '<DEC>', '76', '##55', '[SEP]']

Example 2:
Input sentence: 4<DEC>23245
Label: 1
Token IDs: [101, 1018, 30522, 20666, 19961, 102]
Decoded text: [CLS] 4 <DEC> 23245 [SEP]
Individual tokens: ['[CLS]', '4', '<DEC>', '232', '##45', '[SEP]']

Example 3:
Input sentence: 5<DEC>7655
Label: 0
Token IDs: [101, 1019, 30522, 6146, 24087, 102]
Decoded text: [CLS] 5 <DEC> 7655 [SEP]
Individual tokens: ['[CLS]', '5', '<DEC>', '76', '##55', '[SEP]']
--------------------------------------------------


/home/goreski/miniconda/envs/bert24/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
import string
import itertools

def create_extended_letter_mapping(df, label_column):
    """
    Create a mapping from integer values to unique letters for each column in the dataframe, excluding the label column.
    
    Args:
        df (pd.DataFrame): The input dataframe.
        label_column (str): The name of the label column.
    
    Returns:
        dict: A dictionary where keys are column names and values are mappings from integers to unique letters.
    """
    letter_mapping = {}
    letters = string.ascii_uppercase  # Use uppercase letters A-Z
    
    # Generate combinations of letters (e.g., A, B, ..., Z, AA, AB, ..., ZZ, AAA, AAB, ...)
    max_length = 3  # Adjust this value if needed to handle more unique values
    extended_letters = [''.join(comb) for comb in itertools.chain.from_iterable(itertools.product(letters, repeat=i) for i in range(1, max_length + 1))]
    
    letter_index = 0  # To keep track of the starting index for each column's unique letters
    
    for col in df.columns:
        if col == label_column:
            continue  # Skip the label column
        unique_values = df[col].unique()
        if len(unique_values) > len(extended_letters) - letter_index:
            raise ValueError(f"Too many unique values in column {col} to map to extended letters.")
        
        # Assign a unique subset of letters to this column
        column_letters = extended_letters[letter_index:letter_index + len(unique_values)]
        mapping = {val: column_letters[i] for i, val in enumerate(unique_values)}
        letter_mapping[col] = mapping
        
        # Update the starting index for the next column
        letter_index += len(unique_values)
    
    print("Letter mapping:")
    print(letter_mapping)
    
    return letter_mapping

def transform_discrete_to_letters(df, letter_mapping):
    """
    Transform discrete features from integer encoding to letter encoding, excluding the label column.
    
    Args:
        df (pd.DataFrame): The input dataframe.
        letter_mapping (dict): The mapping from integers to letters for each column.
    
    Returns:
        pd.DataFrame: The transformed dataframe.
    """
    df_transformed = df.copy()
    
    for col, mapping in letter_mapping.items():
        df_transformed[col] = df_transformed[col].map(mapping)
    
    return df_transformed

In [28]:
# Generate the synthetic dataset
df = generate_synthetic_dataset(
    n_samples=cfg.get("n_samples", 100),
    n_continuous_features=cfg.get("n_continuous_features", 0),  # Set to 0 since all features are discrete
    n_discrete_features=cfg.get("n_discrete_features", 15),
    n_classes=cfg.get("n_classes", 2),
    class_distribution=cfg.get("class_distribution", [0.8, 0.2]),
    n_bins=cfg.get("n_bins", 10),
    n_redundant=cfg.get("n_redundant", 0),
    n_noisy=cfg.get("n_noisy", 0),
    class_sep=cfg.get("class_sep", 0.1),
)

# Define the label column
label_column = 'label'

print(df.shape)

# Create extended letter mapping for all columns except the label column
letter_mapping = create_extended_letter_mapping(df, label_column)

# Transform all columns to letter encoding except the label column
df_transformed = transform_discrete_to_letters(df.drop(columns=[label_column]), letter_mapping)

# Add the label column back to the transformed dataframe
df_transformed[label_column] = df[label_column]

# Change structure to "sentence", "label" and "idx"
# All columns except the last one are features and they are concatenated to form a sentence
# The last column is the label
df_transformed['sentence'] = df_transformed.drop(columns=[label_column]).apply(lambda x: ' '.join([str(val) for val in x]), axis=1)

# Reorder columns and add index
df_transformed = df_transformed[['sentence', label_column]]
df_transformed['idx'] = df_transformed.index

# Tokenize the dataset
tokenizer = transformers.AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

# Add a padding token if it doesn't already exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token_id = tokenizer.eos_token_id  # Use EOS token as padding token for GPT-2

# Tokenize sentences
tokenized_dataset = tokenizer(
    df_transformed['sentence'].tolist(),  # Ensure this is a list of strings
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

# Print input sentence and tokenization results
print("\nTokenizer Debug Info:")
print("-" * 50)
# Print first 3 examples
for i in range(min(3, len(df_transformed))):
    print(f"\nExample {i+1}:")
    print(f"Input sentence: {df_transformed['sentence'].iloc[i]}")
    print(f"Label: {df_transformed[label_column].iloc[i]}")
    
    # Get tokenized ids for this example
    tokens = tokenizer.encode(df_transformed['sentence'].iloc[i])
    print(f"Token IDs: {tokens}")
    
    # Decode back to string to verify tokenization
    decoded = tokenizer.decode(tokens)
    print(f"Decoded text: {decoded}")
    
    # Print individual tokens
    tokens_list = tokenizer.convert_ids_to_tokens(tokens)
    print(f"Individual tokens: {tokens_list}")
print("-" * 50)

(100, 16)
Letter mapping:
{'discrete_feature_0': {3: 'A', 1: 'B', 4: 'C', 7: 'D', 5: 'E', 6: 'F', 2: 'G', 9: 'H', 8: 'I', 0: 'J'}, 'discrete_feature_1': {3: 'K', 5: 'L', 4: 'M', 8: 'N', 6: 'O', 7: 'P', 2: 'Q', 1: 'R', 0: 'S', 9: 'T'}, 'discrete_feature_2': {6: 'U', 4: 'V', 5: 'W', 1: 'X', 9: 'Y', 3: 'Z', 2: 'AA', 0: 'AB', 7: 'AC', 8: 'AD'}, 'discrete_feature_3': {4: 'AE', 2: 'AF', 1: 'AG', 6: 'AH', 8: 'AI', 7: 'AJ', 5: 'AK', 3: 'AL', 9: 'AM', 0: 'AN'}, 'discrete_feature_4': {5: 'AO', 3: 'AP', 4: 'AQ', 1: 'AR', 7: 'AS', 6: 'AT', 9: 'AU', 8: 'AV', 2: 'AW', 0: 'AX'}, 'discrete_feature_5': {6: 'AY', 3: 'AZ', 4: 'BA', 5: 'BB', 0: 'BC', 2: 'BD', 8: 'BE', 7: 'BF', 1: 'BG', 9: 'BH'}, 'discrete_feature_6': {7: 'BI', 3: 'BJ', 6: 'BK', 4: 'BL', 1: 'BM', 5: 'BN', 2: 'BO', 0: 'BP', 9: 'BQ', 8: 'BR'}, 'discrete_feature_7': {5: 'BS', 3: 'BT', 2: 'BU', 4: 'BV', 6: 'BW', 7: 'BX', 8: 'BY', 0: 'BZ', 9: 'CA', 1: 'CB'}, 'discrete_feature_8': {3: 'CC', 6: 'CD', 7: 'CE', 5: 'CF', 2: 'CG', 4: 'CH', 0: 'CI', 9

/home/goreski/miniconda/envs/bert24/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
